In [36]:
# !conda install -y -c conda-forge pyspark
# !conda install -y -c conda-forge black

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - black


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    black-22.10.0              |  py310hff52083_0         233 KB  conda-forge
    mypy_extensions-0.4.3      |  py310hff52083_5          11 KB  conda-forge
    pathspec-0.10.1            |     pyhd8ed1ab_0          35 KB  conda-forge
    platformdirs-2.5.2         |     pyhd8ed1ab_1          16 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         294 KB

The following NEW packages will be INSTALLED:

  black              conda-forge/linux-64::black-22.10.0-py310hff52083_0 None
  mypy_extensions    conda-forge/linux-64::mypy_extensions-0.4.3-py310hff52083_5 None
  pathspec           conda-forge/noarch::pathspec-0.10.1-pyhd8ed1ab_0 None
  pla

In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession, DataFrame
from pyspark.conf import SparkConf
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from functools import reduce
import json
import glob
import tempfile

spark = SparkSession.builder \
    .master("local") \
    .getOrCreate()

In [2]:
all_files = all_files = glob.glob("../data/AssignmentData/*.csv")
all_files

['../data/AssignmentData/SMESES000065103320166_2016_07_29_Loan_Data_8396a20b-86e1-4cac-8d51-dbecb46130c6.csv',
 '../data/AssignmentData/SMESES000065103320166_2016_10_31_Loan_Data_5370b5fe-b398-4b40-ace5-2691fc3e1504.csv',
 '../data/AssignmentData/SMESES000065103320166_2017_01_31_Loan_Data_6997f5f3-c2e5-41a4-a038-a5497066fe0e.csv',
 '../data/AssignmentData/SMESES000065103320166_2017_04_30_Loan_Data_52e3e2b7-f8b6-49fd-b482-72a0445cfa9a.csv',
 '../data/AssignmentData/SMESES000065103320166_2017_07_31_Loan_Data_d9629927-17f7-48c0-b1f6-ffbd213308d6.csv',
 '../data/AssignmentData/SMESES000065103320166_2017_10_31_Loan_Data_fb648083-daad-4968-beaa-dfc851ec2b4a.csv',
 '../data/AssignmentData/SMESES000065103320166_2018_01_31_Loan_Data_464b3e03-7988-4c86-a729-a0d31f3e7622.csv',
 '../data/AssignmentData/SMESES000065103320166_2018_04_30_Loan_Data_3a6e2077-7e61-4b77-9dd6-6b4ea1794d05.csv',
 '../data/AssignmentData/SMESES000065103320166_2018_07_31_Loan_Data_af106c4d-a055-49a3-8f62-fede0acb7ce9.csv',
 

In [44]:
asset_columns = {'Pool Cut-off Date':TimestampType(),
'Pool Identifier':StringType(),
'Loan Identifier':StringType(),
'Originator':StringType(),
'Servicer Identifier':StringType(),
'Servicer Name':StringType(),
'Borrower Identifier':StringType(),
'Group Company Identifier':StringType(),
'Country':StringType(),
'Postcode':StringType(),
'Geographic Region':StringType(),
'Obligor Legal Form / Business Type':StringType(),
'Obligor Incorporation Date':TimestampType(),
'Obligor is a Customer since?':TimestampType(),
'Customer segment':StringType(),
'Borrower Basel III Segment':StringType(),
'Originator Affiliate?':BooleanType(),
'Obligor Tax Code':StringType(),
'Asset Type':StringType(),
'Seniority':StringType(),
'Total credit limit granted to the loan':DoubleType(),
'Total Credit Limit Used':DoubleType(),
'Syndicated':BooleanType(),
'Bank Internal Rating':DoubleType(),
'Last Internal Obligor Rating Review':TimestampType(),
'S&P Public Rating (equivalent)':StringType(),
'Moody\'s Public Rating (equivalent)':StringType(),
'Fitch Public Rating (equivalent)':StringType(),
'Dominion Bond Rating Service (DBRS) Public Rating (equivalent)':StringType(),
'Other Public Rating':StringType(),
'Bank Internal Loss Given Default (LGD) Estimate':DoubleType(),
'Bank Internal Loss Given Default (LGD) Estimate (Down-Turn)':DoubleType(),
'S&P Industry Code':StringType(),
'Moody\'s Industry Code':StringType(),
'Fitch Industry Code':StringType(),
'NACE Industry Code':StringType(),
'Other Industry Code':StringType(),
'Borrower deposit amount':DoubleType(),
'Borrower deposit currency':StringType(),
'Loan Origination Date':TimestampType(),
'Final Maturity Date':TimestampType(),
'Loan Denomination Currency':StringType(),
'Loan Hedged':BooleanType(),
'Original Loan Balance':DoubleType(),
'Current Balance':DoubleType(),
'Securitised Loan Amount':DoubleType(),
'Purpose':StringType(),
'Principal Payment Frequency':StringType(),
'Interest Payment Frequency':StringType(),
'Maximum Balance':DoubleType(),
'Weighted Average Life':DoubleType(),
'Amortization Type':StringType(),
'Regular Principal Instalment':DoubleType(),
'Regular Interest Instalment':DoubleType(),
'Type of Loan':StringType(),
'Balloon Amount':DoubleType(),
'Next Payment Date':TimestampType(),
'Payment type':StringType(),
'Prepayment Penalty':DoubleType(),
'Principal Grace Period End Date':TimestampType(),
'Interest Grace Period End Date':TimestampType(),
'Current Interest Rate':DoubleType(),
'Interest Cap Rate':DoubleType(),
'Interest Floor Rate':DoubleType(),
'Interest Rate Type':StringType(),
'Current Interest Rate Index':StringType(),
'Current Interest Rate Margin':DoubleType(),
'Revision Margin 1':DoubleType(),
'Interest Revision Date 1':TimestampType(),
'Revision Margin 2':DoubleType(),
'Interest Revision Date 2':TimestampType(),
'Revision Margin 3':DoubleType(),
'Interest Revision Date 3':TimestampType(),
'Revised Interest Rate Index':StringType(),
'Final Margin':DoubleType(),
'Interest Reset Period':StringType(),
'Turnover of Obligor':DoubleType(),
'Equity':DoubleType(),
'Total Liabilities (excluding Equity)':DoubleType(),
'Short Term Financial Debt':DoubleType(),
'Commercial Liabilities':DoubleType(),
'Long Term Debt':DoubleType(),
'Financial Expenses':DoubleType(),
'Earnings Before Interest, Taxes, Depreciation and Amortisation (EBITDA)':DoubleType(),
'Earnings Before Interest, Taxes (EBIT)':DoubleType(),
'Net Profit':DoubleType(),
'Number of Employees':DoubleType(),
'Currency of Financials':StringType(),
'Date of Financials':TimestampType(),
'Interest Arrears Amount':DoubleType(),
'Number of Days in Interest Arrears':DoubleType(),
'Principal Arrears Amount':DoubleType(),
'Number of Days in Principal Arrears':DoubleType(),
'Loan Entered Arrears':DoubleType(),
'Days in Arrears Prior':DoubleType(),
'Default or Foreclosure on the loan per the transaction definition':BooleanType(),
'Default or Foreclosure on the loan per Basel III definition':BooleanType(),
'Reason for Default (Basel II definition)':StringType(),
'Default Date':TimestampType(),
'Default Amount':DoubleType(),
'Bank Internal Rating Prior to Default':DoubleType(),
'Legal Proceedings Start Date':TimestampType(),
'Cumulative Recoveries':DoubleType(),
'Recovery Source':StringType(),
'Work-out Process Started':TimestampType(),
'Work-out Process Complete':BooleanType(),
'Allocated Losses':DoubleType(),
'Redemption Date':TimestampType(),
'Date Loss Allocated':TimestampType(),
'Real Estate Sale Price':DoubleType(),
'Total Proceeds from Other Collateral or Guarantees':DoubleType(),
'Date of End of Work-out':TimestampType(),
'Foreclosure Cost':DoubleType()}

In [13]:
# Single file will not have all the columns specified in the DTS. Create an ad-hoc schema on the fly.
def get_file_schema(df,master_cols):
    schema = []
    for col_name in df.columns:
        if col_name.endswith("_reason"):
            schema.append(StructField(col_name, StringType(), True))
        else:
            root_col_name = col_name.split("_")[0]
            col_type = master_cols.get(root_col_name, False)
            if col_type:
                schema.append(StructField(col_name, col_type, True))
    return StructType(schema)

In [6]:
# Find List columns and change numerical value to literal
mapper_dict={
    "Obligor Legal Form / Business Type":{1:"Public Company",2:"Limited Company",3:"Partnership",4:"Individul",5:"Other"},
    "Customer Segment":{1:"Medium",2:"Small",3:"Micro",4:"Other"},
    "Borrower Basel III Segment":{1:"Corporate",2:"SME treated as Corporate",3:"Retail",4:"Other"},
    "Asset Type":{1: 'Loan',
                2: 'Guarantee',
                3: 'Promissory Notes',
                4: 'Participation Rights',
                5: 'Overdraft',
                6: 'Letter of Credit',
                7: 'Working Capital Facility',
                8: 'Other'},
    "Seniority":{1: 'Senior Secured',
                2: 'Senior Unsecured',
                3: 'Junior',
                4: 'Junior Unsecured',
                5: 'Other'},
    "Purpose":{1: 'Purchase',
            2: 'Re-mortgage',
            3: 'Renovation',
            4: 'Equity release',
            5: 'Construction Real Estate',
            6: 'Construction Other',
            7: 'Debt consolidation',
            8: 'Re-mortgage with Equity Release',
            9: 'Re-mortgage on Different Terms',
            10: 'Combination Mortgage',
            11: 'Investment Mortgage',
            12: 'Working Capital',
            13: 'Other'},
    "Principal Payment Frequency":{1: 'Monthly',
                                2: 'Quarterly',
                                3: 'Semi annually',
                                4: 'Annual',
                                5: 'Bullet',
                                6: 'Other'},
    "Interest Payment Frequency":{1: 'Linear',
                            2: 'French',
                            3: 'Fix Amortisation Schedule',
                            4: 'Bullet',
                            5: 'Partial Bullet',
                            6: 'Revolving',
                            7: 'Other'},
    "Type of Loan":{1: 'Term', 2: 'Revolving Credit Line', 3: 'Other'},
    "Payment type":{1: 'Direct Debit', 2: 'Standing Order', 3: 'Cheque', 4: 'Cash', 5: 'Other'},
    "Interest Rate Type":{1: 'Floating rate loan for life',
                        2: "Floating rate loan linked to Libor, Euribor, BoE reverting to the Bank's SVR, ECB reverting to Bank’s SVR",
                        3: 'Fixed rate loan for life',
                        4: 'Fixed with future periodic resets',
                        5: 'Fixed rate loan with compulsory future switch to floating',
                        6: 'Capped',
                        7: 'Discount',
                        8: 'Switch Optionality',
                        9: 'Borrower Swapped',
                        10: 'Other'},
    "Current Interest Rate Index":{1: '1 month LIBOR',
                                2: '1 month EURIBOR',
                                3: '3 month LIBOR',
                                4: '3 month EURIBOR',
                                5: '6 month LIBOR',
                                6: '6 month EURIBOR',
                                7: '12 month LIBOR',
                                8: '12 month EURIBOR',
                                9: 'BoE Base Rate',
                                10: 'ECB Base Rate',
                                11: 'Standard Variable Rate',
                                12: 'Other'},
    "Interest Reset Period":{1: 'Annual',
                            2: 'Semi-annual',
                            3: 'Quarterly',
                            4: 'Monthly',
                            5: 'Not apply',
                            6: 'Other'},
    "Reason for Default (Basel II definition)":{1: 'Bankruptcy / Insolvency',
                                            2: 'Failure to Pay',
                                            3: 'Breach of Terms',
                                            4: 'Other'},
    "Recovery Source":{1: 'Bankruptcy / Insolvency',
                        2: 'Failure to Pay',
                        3: 'Breach of Terms',
                        4: 'Other'},
    "Recovery Source":{1: 'Liquidation of Collateral',
                        2: 'Enforcement of Guarantees',
                        3: 'Additional Lending',
                        4: 'Cash Recoveries',
                        5: 'Mixed',
                        6: 'Other'},
}

In [7]:
# Change NoData value with NA and create a "*_reason" column with literal explanation
# Value	Reason	Example
# 1	Data not collected as not required by the underwriting criteria	ND,1
# 2	Data collected at application but not loaded in the reporting system at completion	ND,2
# 3	Data collected at application but loaded in a separate system from the reporting one	ND,3
# 4	Data collected but will only be available from YYYY-MM	ND,4,YYYY-MM
# 5	Not relevant at the present time	ND,5
# 6	Not applicable for the jurisdiction (see table in 'Assets' and 'Collateral' sheets)	ND,6

def replace_no_data(df):
    no_data_explanations = {'1':"Data not collected as not required by the underwriting criteria",
                        '2':"Data collected at application but not loaded in the reporting system at completion",
                        '3':"Data collected at application but loaded in a separate system from the reporting one",
                        '4':"Data collected but will only be available in later releases",
                        '5':"Not relevant at the present time",
                        '6':"Not applicable for the jurisdiction",
    }
    col_names_to_drop = []
    for col_name in df.columns:
        if sum([str(v).startswith("ND") for v in df[col_name].unique()])>0:
            col_names_to_drop.append(col_name)
            value_col_name = f"{col_name}_value"
            reason_col_name = f"{col_name}_reason"
            df[[value_col_name, reason_col_name]] = df[col_name].str.split(",", expand=True)[[0,1]]
            df[reason_col_name]=df[reason_col_name].map(no_data_explanations).astype("str")
    df.drop(col_names_to_drop,axis=1,inplace=True)
    df.replace("ND", np.nan, inplace=True)
    # df.replace("ND", None, inplace=True)
    # df.fillna(np.nan, inplace=True)
    return df

In [40]:
# Conver Date columns into datetime
def convert_date_cols(df):
    columns = df.columns.tolist()
    col_names=["Pool Cut-off Date",
                "Obligor Incorporation Date",
                "Obligor is a Customer since?",
                "Last Internal Obligor Rating Review",
                "Loan Origination Date",
                "Final Maturity Date",
                "Next Payment Date",
                "Principal Grace Period End Date",
                "Interest Grace Period End Date",
                "Interest Revision Date 1",
                "Interest Revision Date 2",
                "Interest Revision Date 3",
                "Date of Financials",
                "Default Date",
                "Legal Proceedings Start Date",
                "Work-out Process Started",
                "Redemption Date",
                "Date Loss Allocated",
                "Date of End of Work-out"]
    for col_name in col_names:
        # column could contain ND, so it could be split in *_value and *_reason
        alt_col_name = f"{col_name}_value"
        final_col_name = alt_col_name if df.columns.tolist().count(alt_col_name) > 0 else col_name
        # Column might not be present in df
        if final_col_name in columns:
            df[final_col_name] = pd.to_datetime(df[final_col_name], infer_datetime_format=False, format="%Y-%m-%d", utc=True)
    return df


# Convert Numeric columns into numbers
def convert_numeric_cols(df):
    columns = df.columns.tolist()
    col_names=["Total credit limit granted to the loan",
                "Total Credit Limit Used",
                "Bank Internal Rating",
                "Bank Internal Loss Given Default (LGD) Estimate",
                "Bank Internal Loss Given Default (LGD) Estimate (Down-Turn)",
                "S&P Industry Code",
                "Moody's Industry Code",
                "Fitch Industry Code",
                "Borrower deposit amount",
                "Original Loan Balance",
                "Current Balance",
                "Securitised Loan Amount",
                "Maximum Balance",
                "Weighted Average Life",
                "Regular Principal Instalment",
                "Regular Interest Instalment",
                "Balloon Amount",
                "Prepayment Penalty",
                "Current Interest Rate",
                "Interest Cap Rate",
                "Interest Floor Rate",
                "Current Interest Rate Margin",
                "Revision Margin 1",
                "Revision Margin 2",
                "Revision Margin 3",
                "Final Margin",
                "Turnover of Obligor",
                "Equity",
                "Total Liabilities (excluding Equity)",
                "Short Term Financial Debt", 
                "Commercial Liabilities", 
                "Long Term Debt",
                "Financial Expenses",
                "Earnings Before Interest, Taxes, Depreciation and Amortisation (EBITDA)", 
                "Earnings Before Interest, Taxes (EBIT)", 
                "Net Profit",
                "Number of Employees",
                "Interest Arrears Amount",
                "Number of Days in Interest Arrears",
                "Principal Arrears Amount",
                "Number of Days in Principal Arrears",
                "Loan Entered Arrears",
                "Days in Arrears Prior",
                "Default Amount",
                "Bank Internal Rating Prior to Default",
                "Cumulative Recoveries",
                "Allocated Losses",
                "Real Estate Sale Price",
                "Total Proceeds from Other Collateral or Guarantees",
                "Foreclosure Cost"]
    for col_name in col_names:
        # column could contain ND, so it could be split in *_value and *_reason
        alt_col_name = f"{col_name}_value"
        final_col_name = alt_col_name if df.columns.tolist().count(alt_col_name) > 0 else col_name
        # Column might still not be present in df
        if final_col_name in columns:
            df[final_col_name] = pd.to_numeric(df[final_col_name]).astype('float64')
    return df

In [45]:
list_dfs = []
for csv_f in all_files:
    # with open(csv_f,'rb') as f:
    #     content = f.readlines()[1:]
    #     with tempfile.NamedTemporaryFile() as tf:
    #         tf.writelines(content)
    #         df = spark.read.csv(tf.name, header=True)
    df = pd.read_csv(csv_f, header=1)
    df.columns = [c.strip() for c in df.columns.tolist()]
    df_1 = df.replace(mapper_dict)
    df_2 = replace_no_data(df_1)
    df_3 = convert_date_cols(df_2)
    df_4 = convert_numeric_cols(df_3)
    # Replace Y/N with True/False for boolean columns
    df_4.replace({'N': False, 'Y':True}, inplace=True)
    curr_schema = get_file_schema(df_4, asset_columns)
    try:
        list_dfs.append(spark.createDataFrame(df_4, schema=curr_schema))
    except Exception as e:
        print(f"File:{csv_f}")
        print(e)
        break

assets_df = reduce(DataFrame.union, list_dfs)
# assets_df = pd.concat(list_dfs,ignore_index=True)

/opt/conda/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


File:../data/AssignmentData/SMESES000065103320166_2016_07_29_Loan_Data_8396a20b-86e1-4cac-8d51-dbecb46130c6.csv
field Short Term Financial Debt: DoubleType() can not accept object 0 in type <class 'int'>


TypeError: reduce() of empty iterable with no initial value

In [37]:
test_csv = '../data/AssignmentData/SMESES000065103320166_2017_01_31_Loan_Data_6997f5f3-c2e5-41a4-a038-a5497066fe0e.csv'
df = pd.read_csv(test_csv, header=1)
df_1 = df.replace(mapper_dict)
df_2 = replace_no_data(df_1)
df_3 = convert_date_cols(df_2)
df_4 = convert_numeric_cols(df_3)
df_4.replace({'N': False, 'Y':True}, inplace=True)

In [39]:
df["Short Term Financial Debt "].unique()

array([0])

In [42]:
"Short Term Financial Debt ".strip()

'Short Term Financial Debt'